# Detecting pneumonia

### First some imports

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import sys
from glob import glob

### Specific tensorflow + keras imports

In [59]:
#import tensorflow as tf
import tensorflow.compat.v1 as tf

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras import applications

from tensorflow.python.framework import graph_util
from tensorflow.python.platform import gfile

tf.disable_v2_behavior()


## Data

In [8]:
# Input image size
def get_model_config():
  inception_url = "http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz"
  bottleneck_tensor_name = "pool_3/_reshape:0"
  bottleneck_tensor_size = 2048
  input_width = 299 
  input_height = 299
  input_depth = 3
  resized_input_tensor_name = "Mul:0"
  model_file_name = "classify_image_graph_def.pb"
  input_mean = 128
  input_std = 128

  return {
      "inception_url": inception_url,
      "bottleneck_tensor_name": bottleneck_tensor_name,
      "bottleneck_tensor_size": bottleneck_tensor_size,
      "input_width": input_width,
      "input_height": input_height,
      "input_depth": input_depth,
      "resized_input_tensor_name": resized_input_tensor_name,
      "model_file_name": model_file_name,
      "input_mean": input_mean,
      "input_std": input_std,
  }input_shape = (28,28,1)

In [56]:

def get_image_files(image_dir):
  fs = glob("{}/*.jpeg".format(image_dir))
  fs = [os.path.basename(filename) for filename in fs]
  return sorted(fs)

def create_image_lists(image_dir):
  result = {}

  training_images = []
  testing_images = []
  validation_images = []

  for category in ["train", "test", "val"]:
    category_path = os.path.join(image_dir, category)
    try:
      bins = next(os.walk(category_path))[1]
    except StopIteration:
      sys.exit("ERROR: Missing either train/test/val folders in image_dir")
    for diagnosis in bins:
      bin_path = os.path.join(category_path, diagnosis)
      if category == "train":
        training_images.append(get_image_files(bin_path))
      if category == "test":
        testing_images.append(get_image_files(bin_path))
      if category == "val":
        validation_images.append(get_image_files(bin_path))

  for diagnosis in bins:
    result[diagnosis] = {
      "training": training_images[bins.index(diagnosis)],
      "testing": testing_images[bins.index(diagnosis)],
      "validation": validation_images[bins.index(diagnosis)],
    }
  return result

# Return graph of network and bottleneck tensor
def create_model_graph(model_info, model_dir):
  with tf.Graph().as_default() as graph:
    model_path = os.path.join(model_dir, model_info["model_file_name"])
    with gfile.FastGFile(model_path, "rb") as f:
      graph_def = tf.GraphDef()
      graph_def.ParseFromString(f.read())
      bottleneck_tensor, resized_input_tensor = (tf.import_graph_def(
        graph_def,
        name="",
        return_elements=[
          model_info["bottleneck_tensor_name"],
          model_info["resized_input_tensor_name"],
        ]))
  return graph, bottleneck_tensor, resized_input_tensor

# Returns tensors to feed jpeg data into
def decode_jpeg(input_width, input_height, input_depth, input_mean, input_std):
  jpeg_data = tf.placeholder(tf.string, name="DecodeJPEGInput")
  decoded_image = tf.image.decode_jpeg(jpeg_data, channels=input_depth)
  decoded_image_as_float = tf.cast(decoded_image, dtype=tf.float32)
  decoded_image_4d = tf.expand_dims(decoded_image_as_float, 0)
  resize_shape = tf.stack([input_height, input_width])
  resize_shape_as_int = tf.cast(resize_shape, dtype=tf.int32)
  resized_image = tf.image.resize_bilinear(decoded_image_4d,
                                           resize_shape_as_int)
  offset_image = tf.subtract(resized_image, input_mean)
  mul_image = tf.multiply(offset_image, 1.0 / input_std)
  return jpeg_data, mul_image


In [36]:

def get_image_files(image_dir):
  fs = glob("{}/*.jpeg".format(image_dir))
  fs = [os.path.basename(filename) for filename in fs]
  return sorted(fs)

def create_image_lists(image_dir):
  result = {}

  training_images = []
  testing_images = []
  validation_images = []

  for category in ["train", "test", "val"]:
    category_path = os.path.join(image_dir, category)
    try:
      bins = next(os.walk(category_path))[1]
    except StopIteration:
      sys.exit("ERROR: Missing either train/test/val folders in image_dir")
    for diagnosis in bins:
      bin_path = os.path.join(category_path, diagnosis)
      if category == "train":
        training_images.append(get_image_files(bin_path))
      if category == "test":
        testing_images.append(get_image_files(bin_path))
      if category == "val":
        validation_images.append(get_image_files(bin_path))

  for diagnosis in bins:
    result[diagnosis] = {
      "training": training_images[bins.index(diagnosis)],
      "testing": testing_images[bins.index(diagnosis)],
      "validation": validation_images[bins.index(diagnosis)],
    }
  return result



create_image_lists("./new_files copy")

{'BACTERIA': {'training': ['BACTERIA-1008087-0001.jpeg',
   'BACTERIA-1025587-0001.jpeg',
   'BACTERIA-1027194-0001.jpeg',
   'BACTERIA-1033441-0001.jpeg',
   'BACTERIA-103798-0001.jpeg',
   'BACTERIA-103798-0002.jpeg',
   'BACTERIA-1039608-0001.jpeg',
   'BACTERIA-1040986-0001.jpeg',
   'BACTERIA-1041881-0001.jpeg',
   'BACTERIA-1041881-0002.jpeg',
   'BACTERIA-104960-0001.jpeg',
   'BACTERIA-1065620-0001.jpeg',
   'BACTERIA-1065620-0002.jpeg',
   'BACTERIA-1069837-0001.jpeg',
   'BACTERIA-1069837-0002.jpeg',
   'BACTERIA-1074715-0001.jpeg',
   'BACTERIA-1076722-0001.jpeg',
   'BACTERIA-1083680-0001.jpeg',
   'BACTERIA-1083680-0002.jpeg',
   'BACTERIA-1083680-0003.jpeg',
   'BACTERIA-1083680-0004.jpeg',
   'BACTERIA-1083680-0005.jpeg',
   'BACTERIA-1083680-0006.jpeg',
   'BACTERIA-1083680-0007.jpeg',
   'BACTERIA-1083680-0008.jpeg',
   'BACTERIA-1083680-0009.jpeg',
   'BACTERIA-1083680-0010.jpeg',
   'BACTERIA-1083680-0011.jpeg',
   'BACTERIA-1083680-0012.jpeg',
   'BACTERIA-1083680-0

In [62]:
jpeg_data_tensor, decoded_image_tensor = decode_jpeg(
      299, 299, 3, 128, 128)

jpeg_data_tensor

<tf.Tensor 'DecodeJPEGInput_2:0' shape=<unknown> dtype=string>

## Define model

In [9]:
# Placeholder model
model_p = Sequential()


model_p.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=input_shape))
model_p.add(Conv2D(32, kernel_size=3, activation='relu'))
model_p.add(Flatten())
model_P.add(Dense(10, activation='softmax'))

In [25]:
model = applications.inception_resnet_v2.InceptionResNetV2(
    include_top=False, weights='imagenet', input_shape=(299, 299, 3), pooling='avg')

In [24]:
model.summary()

for layer in model.layers:
    layer.trainable = False

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_204 (Conv2D)             (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_204 (BatchN (None, 149, 149, 32) 96          conv2d_204[0][0]                 
__________________________________________________________________________________________________
activation_204 (Activation)     (None, 149, 149, 32) 0           batch_normalization_204[0][0]    
________________________________________________________________________________

## Compile model

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Fit model

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)